In [2]:
import nltk
import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
import pickle

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


In [3]:
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...


True

In [1]:
import os
import json

os.chdir('C:/Users/sagar/my-chatbot')

with open('data/intents.json') as file:
    intents = json.load(file)

print("✅ Intents loaded successfully!")
print(json.dumps(intents, indent=2)[:500])







✅ Intents loaded successfully!
{
  "intents": [
    {
      "tag": "greeting",
      "patterns": [
        "Hi",
        "Hello",
        "Hey",
        "Good morning"
      ],
      "responses": [
        "Hello!",
        "Hi there!",
        "Greetings!"
      ]
    },
    {
      "tag": "goodbye",
      "patterns": [
        "Bye",
        "See you later",
        "Goodbye"
      ],
      "responses": [
        "Bye!",
        "See you later!",
        "Goodbye!"
      ]
    }
  ]
}


In [4]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import random

lemmatizer = WordNetLemmatizer()

words = []
tags = []
documents = []

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
    if intent['tag'] not in tags:
        tags.append(intent['tag'])

# Lemmatize, lower, and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w.isalnum()]
words = sorted(set(words))
tags = sorted(set(tags))

print(f"✅ Words: {words}")
print(f"✅ Tags: {tags}")


✅ Words: ['bye', 'good', 'goodbye', 'hello', 'hey', 'hi', 'later', 'morning', 'see', 'you']
✅ Tags: ['goodbye', 'greeting']


In [8]:
import nltk
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       tagge

True

In [11]:
import json
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Load intents
with open('data/intents.json') as file:
    intents = json.load(file)

# Data structures
words = []
tags = []
documents = []

# Tokenize, collect words/tags
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
    if intent['tag'] not in tags:
        tags.append(intent['tag'])

# Lemmatize & clean
words = [lemmatizer.lemmatize(w.lower()) for w in words if w.isalnum()]
words = sorted(set(words))
tags = sorted(set(tags))

print(f"✅ Words (vocabulary): {words}")
print(f"✅ Tags (classes): {tags}")

# Bag-of-Words + one-hot labels
training = []
output_empty = [0] * len(tags)

for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[tags.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

print(f"✅ train_x shape: {train_x.shape}")
print(f"✅ train_y shape: {train_y.shape}")

# Model
model = Sequential()
model.add(Dense(8, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(len(train_y[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(train_x, train_y, epochs=200, batch_size=8, verbose=1)

# Save
model.save('chatbot_model.h5')
pickle.dump(words, open('word_list.pkl', 'wb'))
pickle.dump(tags, open('tag_list.pkl', 'wb'))

print("✅ Model and data saved!")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\users\sagar\my-chatbot\venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ Words (vocabulary): ['bye', 'good', 'goodbye', 'hello', 'hey', 'hi', 'later', 'morning', 'see', 'you']
✅ Tags (classes): ['goodbye', 'greeting']
✅ train_x shape: (7, 10)
✅ train_y shape: (7, 2)
Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4286 - loss: 0.7005
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.4286 - loss: 0.6990
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.4286 - loss: 0.6975
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.4286 - loss: 0.6961
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.4286 - loss: 0.6947
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.4286 - loss: 0.6933
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5714 - loss: 0.6918
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5714 - loss: 0.6904
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.5714 - loss: 0.6890
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step

✅ Model and data saved!


In [1]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

# Load saved artifacts
lemmatizer = WordNetLemmatizer()
model = load_model('chatbot_model.h5')
words = pickle.load(open('word_list.pkl', 'rb'))
tags = pickle.load(open('tag_list.pkl', 'rb'))

with open('data/intents.json') as file:
    intents = json.load(file)

# Helper: bag-of-words
def bag_of_words(sentence, words):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words if word.isalnum()]
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

# Helper: predict intent
def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.5
    results = [(i, r) for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": tags[r[0]], "probability": str(r[1])} for r in results]

# Helper: get response
def get_response(intents_list, intents_json):
    if intents_list:
        tag = intents_list[0]['intent']
        for i in intents_json['intents']:
            if i['tag'] == tag:
                return random.choice(i['responses'])
    return "I don't understand. Can you rephrase?"

# Main chat loop
print("Chatbot is running! (type 'quit' to stop)")
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    ints = predict_class(message)
    res = get_response(ints, intents)
    print("Bot:", res)


Chatbot is running! (type 'quit' to stop)


You:  hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
Bot: Greetings!


You:  quit


In [2]:
pip install flask


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\sagar\my-chatbot\venv\scripts\python.exe -m pip install --upgrade pip' command.
